In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
import torch.nn as nn

(train_images, train_labels),(test_images, test_labels)=mnist.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)
train_images=train_images.reshape([-1,784]).astype('float32')/255.0
test_images=test_images.reshape([-1,784]).astype('float32')/255.0
train_labels=tf.one_hot(train_labels, depth=10) # size 10짜리 one-hot encoding 만듦
test_labels=tf.one_hot(test_labels, depth=10)
batch_size=1000
train_data=tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_data=train_data.repeat().shuffle(5000).batch(batch_size) # 데이터를 shuffle 하여야 고른 분포를 가짐

nH1=512
nH2=512
nH3=512
nH4=512
nH5=512

class class_SoftmaxNN_LC(nn.Module): # class 는 항상 pytorch로 만들었기 때문에
  def __init__(self):
    self.W_1=tf.Variable(tf.random.normal([784,nH1]))
    self.b_1=tf.Variable(tf.random.normal([nH1]))
    self.W_2=tf.Variable(tf.random.normal([nH1,nH2]))
    self.b_2=tf.Variable(tf.random.normal([nH2]))
    self.W_3=tf.Variable(tf.random.normal([nH2,nH3]))
    self.b_3=tf.Variable(tf.random.normal([nH3]))
    self.W_4=tf.Variable(tf.random.normal([nH3,nH4]))
    self.b_4=tf.Variable(tf.random.normal([nH4]))
    self.W_5=tf.Variable(tf.random.normal([nH4,nH5]))
    self.b_5=tf.Variable(tf.random.normal([nH5]))
    self.W_o=tf.Variable(tf.random.normal([nH5,10]))
    self.b_o=tf.Variable(tf.random.normal([10]))
  def __call__(self,x):
    H_1=tf.nn.relu(tf.matmul(x,self.W_1)+self.b_1)
    H_2=tf.nn.relu(tf.matmul(H_1,self.W_2)+self.b_2)
    H_3=tf.nn.relu(tf.matmul(H_2,self.W_3)+self.b_3)
    H_4=tf.nn.relu(tf.matmul(H_3,self.W_4)+self.b_4)
    H_5=tf.nn.relu(tf.matmul(H_4,self.W_5)+self.b_5)
    Out=tf.matmul(H_5,self.W_o)+self.b_o
    return Out

model_SoftmaxNN_LC=class_SoftmaxNN_LC()
optimizer=tf.optimizers.Adam(0.01)

def cost_Softmax(x,y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=x,labels=y))

def train_optimization(MNIST_images,MNIST_labels):
  with tf.GradientTape() as g:
    prediction=model_SoftmaxNN_LC(MNIST_images)
    cost=cost_Softmax(prediction,MNIST_labels)
  gradients=g.gradient(cost,vars(model_SoftmaxNN_LC).values())
  tf.optimizers.Adam(0.01).apply_gradients(zip(gradients,vars(model_SoftmaxNN_LC).values()))

for epoch in range(1,11):
  for step, (batch_images,batch_labels) in enumerate(train_data.take(60),1):
    train_optimization(batch_images,batch_labels)
    if step%10==0:
      pred=tf.nn.softmax(model_SoftmaxNN_LC(batch_images))
      accuracy=tf.reduce_mean(tf.cast(tf.equal(pred,batch_labels),tf.float32))
      print('Epoch: {} \t Step: {} \t Accuracy: {}'.format(epoch,step,accuracy))

print('Testing')
test_model=tf.nn.softmax(model_SoftmaxNN_LC(test_images))
test_accuracy=tf.reduce_mean(tf.cast(tf.equal(pred,batch_labels),tf.float32))
print('Test Accuracy:', test_accuracy.numpy())

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)
Epoch: 1 	 Step: 10 	 Accuracy: 0.8917999863624573
Epoch: 1 	 Step: 20 	 Accuracy: 0.9458000063896179
Epoch: 1 	 Step: 30 	 Accuracy: 0.9531999826431274
Epoch: 1 	 Step: 40 	 Accuracy: 0.9656000137329102
Epoch: 1 	 Step: 50 	 Accuracy: 0.968999981880188
Epoch: 1 	 Step: 60 	 Accuracy: 0.9783999919891357
Epoch: 2 	 Step: 10 	 Accuracy: 0.9746000170707703
Epoch: 2 	 Step: 20 	 Accuracy: 0.9807999730110168
Epoch: 2 	 Step: 30 	 Accuracy: 0.9824000000953674
Epoch: 2 	 Step: 40 	 Accuracy: 0.9842000007629395
Epoch: 2 	 Step: 50 	 Accuracy: 0.98580002784729
Epoch: 2 	 Step: 60 	 Accuracy: 0.9883999824523926
Epoch: 3 	 Step: 10 	 Accuracy: 0.991599977016449
Epoch: 3 	 Step: 20 	 Accuracy: 0.9846000075340271
Epoch: 3 	 Step: 30 	 Accuracy: 0.9911999702453613
Epoch: 3 	 Step: 40 	 Accuracy: 0.9887999892234802
Epoch: 3 	 Step: 50 	 Accuracy: 0.9890000224113464
Epoch: 3 	 Step: 60 	 